In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# if using a Jupyter notebook, includue:
%matplotlib inline

In [2]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
chosenSpacing=(1.5,1.5,1.5) # chosen uniform target spacing

"""
looking through all valid spacings (if it si invalid it goes below 0)
and displaying minimal maximal and rounded mean spacing and median
in my case median and mean values are close - and using the median values will lead to a bit less interpolations later
"""
spacingDict={}
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
    for addedKey in ['_spac_x','_spac_y','_spac_z']:   
        colName = keyWord+addedKey
        liist = list(filter(lambda it: it>0 ,df[colName].to_numpy() ))
        minn=np.min(liist)                
        maxx=np.max(liist)
        meanRounded = round((minn+maxx)/2,1)
        medianRounded = round(np.median(liist),1)
        spacingDict[colName]=(minn,maxx,meanRounded,medianRounded)

spacingDict



{'t2w_spac_x': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_y': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_z': (2.200000060372773, 5.000000066297424, 3.6, 3.0),
 'adc_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'cor_spac_x': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_y': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_z': (0.9999999999999636, 4.49999998188102, 2.7, 3.0),
 'hbv_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'sag_spac_x': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_y': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_z': (2.999999882111563, 4.000404550206146, 3.5, 3.6)}

In [25]:
df[['t2w_spac_x', 'hbv_spac_x']]


,t2w_spac_x,hbv_spac_x
0,0.28125,1.754386
1,0.30000,2.000000
2,0.50000,2.000000
3,0.50000,2.000000
4,0.30000,2.000000
...,...,...
1495,0.50000,2.000000
1496,0.30000,2.000000
1497,0.30000,2.000000
1498,0.62500,1.857143


In [21]:
from KevinSR import mask_interpolation, SOUP_GAN
import os
import numpy as np
from scipy import ndimage, interpolate
from scipy.ndimage import zoom
from KevinSR import SOUP_GAN
import monai
import SimpleITK as sitk
from copy import deepcopy
import tensorflow as tf
#from numba import cuda 

def superSampleinZ(data, currentSpacZ, targetSpacZ):
    """
    perform supersampling in Z direction
    currentSpac, targetSpac - current and target z spacing - float numbers
    """
    Z_FAC = currentSpacZ/targetSpacZ
    # Call the SR interpolation tool from KevinSR
    thins_gen = SOUP_GAN(data, Z_FAC, 0)
    # device = cuda.get_current_device()
    # device.reset()
    return thins_gen
    
def subsample(image,orig_spacing, targetSpacZ,origSize):
    """
    perform subsampling to targetSpac
    TODO() bicubic interpolation adapted from 
        https://itk.org/Doxygen/html/WikiExamples_2ImageProcessing_2Upsampling_8cxx-example.html
        https://discourse.itk.org/t/resample-produces-blurry-results-when-just-cropping/4473/8
    """ 
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(targetSpacZ)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(image.GetPixelIDValue())
    resample.SetInterpolator(sitk.sitkBSpline)
    
    new_size = tuple([int(origSize[0]*(orig_spacing[0]/targetSpacZ[0])),
                int(origSize[1]*(orig_spacing[1]/targetSpacZ[1])),
                int(origSize[2]*(orig_spacing[2]/targetSpacZ[2]) )  ]  )
    
    print(f"new_size {new_size}")
    resample.SetSize(new_size)

    res = resample.Execute(image)
    print(f"subsampled {res.GetSize()}")
    return res 


def transformSpacingTo(path, targetSpac):
    """
    transforming spacing of the image which path is supplied to target spacing values
    upsampling will be performed with SOUP-GAN  and downsampling will be done via simple itk bicubic interpolation
    """
    image = sitk.ReadImage(path)
    spac = image.GetSpacing()
    origSpac=spac
    spacGoal=list(deepcopy(spac))    
    origSize= image.GetSize()
    data = sitk.GetArrayFromImage(image)
    data = np.moveaxis(data, 0, 2)
    
    anySuperSampled = False
    for axis in [0,1,2]:
        #tuple are immutable ...
        spacGoal[axis]=targetSpac[axis]
# abs        print(f"spac {spac} axis {axis} spacing goal {spacGoal} ")
#         if(targetSpac[axis]>spac[axis]):
#             #if target spacing is bigger we need to subsample

#             #overwriting image by resampled one
#             image=subsample(image, tuple(spacGoal))
#             pass
        print(f"axis {axis}  targetSpac[axis] ${targetSpac[axis]} spac[axis] {spac[axis]}")
        if(targetSpac[axis]<spac[axis]):
            anySuperSampled=True
            #we need to perform supersampling - now as supersampling can be performed only in z direction
            #we need to rotate image accordingly       
            if(axis==0):
                print(f" start data shape {data.shape}")                
                transformed = np.moveaxis(data, 0, 2)
                print(f" transformed data shape {transformed.shape} in axis 0")
                data=superSampleinZ(transformed, spac[axis], targetSpac[axis])
                print(f" supersampled data shape {data.shape} in axis 0")
                #getting back original shape
                data = np.moveaxis(data, 2, 0)
                print(f" tr back data shape {data.shape} in axis 0")
                
            if(axis==1):
                transformed = np.moveaxis(data, 1, 2)
                print(f" transformed data shape {transformed.shape} in axis 1")                                
                data=superSampleinZ(transformed, spac[axis], targetSpac[axis])
                print(f" supersampled data shape {transformed.shape} in axis 1")
                #getting back original shape
                data = np.moveaxis(data, 2, 1)   
                print(f" tr back data shape {data.shape} in axis 1")
                
            if(axis==2):
                # no need to transform
                data=superSampleinZ(data, spac[axis], targetSpac[axis])

    if(anySuperSampled):
        print(f"data shape after ifs {data.shape}")
        image1 = sitk.GetImageFromArray(data)
        print(f"data image size  after ifs {image1.GetSize()}")        
        image1.SetSpacing(tuple(spacGoal)) #updating spacing
        image1.SetOrigin(image.GetOrigin())
        image1.SetDirection(image.GetDirection())
        image = image1
            #in case of equality we do nothing
    #image=subsample(image,origSpac, tuple(spacGoal),origSize)
    return image






In [20]:
path ='/home/sliceruser/data/orig/10019/10019_1000019_t2w.mha'
imageOrig = sitk.ReadImage(path)
orig_spac = imageOrig.GetSpacing()
print(orig_spac)
print(imageOrig.GetSize())
data = sitk.GetArrayFromImage(imageOrig)
print(data.shape)

(0.5, 0.5, 2.999999996213984)
(384, 384, 19)
(19, 384, 384)


In [22]:
spacc=(0.3,0.3, 2)
imagee=transformSpacingTo(path,spacc)
imagee.GetSpacing()

axis 0  targetSpac[axis] $0.3 spac[axis] 0.5
 start data shape (384, 384, 19)
 transformed data shape (384, 19, 384) in axis 0
1/1 [==============================] - 14s 14s/step
 supersampled data shape (384, 19, 640) in axis 0
 tr back data shape (640, 384, 19) in axis 0
axis 1  targetSpac[axis] $0.3 spac[axis] 0.5
 transformed data shape (640, 19, 384) in axis 1
1/1 [==============================] - 21s 21s/step
 supersampled data shape (640, 19, 384) in axis 1
 tr back data shape (640, 640, 19) in axis 1
axis 2  targetSpac[axis] $2 spac[axis] 2.999999996213984
1/1 [==============================] - 34s 34s/step
data shape after ifs (640, 640, 28)
data image size  after ifs (28, 640, 640)


(0.3, 0.3, 2.0)

In [6]:
testImagePath = '/home/sliceruser/data/exploration/resampleddB.nii.gz'

writer = sitk.ImageFileWriter()
imagee.GetSpacing()
writer.KeepOriginalImageUIDOn()
writer.SetFileName(testImagePath)
writer.Execute(imagee)  

testImagePathB = '/home/sliceruser/data/exploration/resampledd_itk.nii.gz'
imageITKk=subsample(imageOrig, orig_spac,spacc,imageOrig.GetSize())

writer = sitk.ImageFileWriter()
writer.KeepOriginalImageUIDOn()
writer.SetFileName(testImagePathB)
writer.Execute(imageITKk)  


new_size (640, 640, 28)
subsampled (640, 640, 28)


In [7]:
imagee.GetSize()
# all axes (384, 1152, 32)

(576, 640, 32)

In [8]:
# def transform_low_res_spacing_dataset(targetCommonSpacing):
#     """
#     transforms to uniform spacing images ['t2w','adc',,'hbv',  ] but not   'cor' 'sag'
#     """
#     df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
#     for seriesString in ['t2w','adc', 'hbv' ]: 
#         train_patientsPaths=df[seriesString].dropna().astype('str').to_numpy()
#         train_patientsPaths=list(filter(lambda path: len(path)>2 ,train_patientsPaths))
#         for path in train_patientsPaths:
#             image=transformSpacingTo(path, targetCommonSpacing)
#             #overwriting data but changing names
#             newPath=path.replace(".mha","_resampled.mha")
#             writer = sitk.ImageFileWriter()
#             writer.KeepOriginalImageUIDOn()
#             writer.SetFileName(newPath)
#             writer.Execute(image)  


# transform_low_res_spacing_dataset(chosenSpacing)

# def get_spatial_meta(row,colName):
#     """
#     refreshing data about size and spacing
#     """
#     row=row[1]
#     patId=str(row['patient_id'])
#     path=str(row[colName])
#     path=path.replace(".mha","_resampled.mha")
#     if(len(path)>1):
#         image = sitk.ReadImage(path)
#         sizz= ifShortReturnMinus(image.GetSize(),patId,colName )
#         spac= ifShortReturnMinus(image.GetSpacing(),patId,colName)
#         orig= ifShortReturnMinus(image.GetOrigin(),patId,colName)
#         return list(sizz)+list(spac)+list(orig)
#     return [-1,-1,-1,-1,-1,-1,-1,-1,-1]
# for keyWord in ['t2w','adc', 'hbv' ]:    
#     resList=[]
#     with mp.Pool(processes = mp.cpu_count()) as pool:
#         resList=pool.map(partial(get_spatial_meta,colName=keyWord)  ,list(df.iterrows()))    
#     print(type(resList))    
#     df[keyWord+'_sizz_x']= list(map(lambda arr:arr[0], resList))    
#     df[keyWord+'_sizz_y']= list(map(lambda arr:arr[1], resList))    
#     df[keyWord+'_sizz_z']= list(map(lambda arr:arr[2], resList))    

In [ ]:
testImagePath = '/home/sliceruser/data/exploration/resampleddB.nii.gz'
import pandas as pd
import SimpleITK as sitk

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
row= df.iloc[19]
pathT2w= row['t2w']
pathHbv= row['hbv']
pathADC= row['adc']
patht2wLabel= row['reSampledPath']




writer = sitk.ImageFileWriter()
imagee.GetSpacing()
writer.KeepOriginalImageUIDOn()
writer.SetFileName(testImagePath)
writer.Execute(imagee)  

testImagePathB = '/home/sliceruser/data/exploration/resampledd_itk.nii.gz'
imageITKk=subsample(imageOrig, orig_spac,spacc,imageOrig.GetSize())

writer = sitk.ImageFileWriter()
writer.KeepOriginalImageUIDOn()
writer.SetFileName(testImagePathB)
writer.Execute(imageITKk)  


In [3]:
import pandas as pd
import SimpleITK as sitk
from KevinSR import mask_interpolation, SOUP_GAN
import os
import numpy as np
from scipy import ndimage, interpolate
from scipy.ndimage import zoom
from KevinSR import SOUP_GAN
import monai
import SimpleITK as sitk
from copy import deepcopy
import tensorflow as tf

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
row= df.iloc[19]
pathT2w= row['t2w']
pathHbv= row['hbv']
pathADC= row['adc']
patht2wLabel= row['reSampledPath']
testImagePath = '/home/sliceruser/data/exploration/resampledd.mha'

#pathT2w,pathHbv,pathADC,patht2wLabel
def testResample(path, nameToDisp,targetSpac):
    imageOrig = sitk.ReadImage(path)
    origSize= imageOrig.GetSize()
    orig_spacing=imageOrig.GetSpacing()
    spacc=(0.3,0.3, 2)
    new_size = tuple([int(origSize[0]*(orig_spacing[0]/targetSpac[0])),
                    int(origSize[1]*(orig_spacing[1]/targetSpac[1])),
                    int(origSize[2]*(orig_spacing[2]/targetSpac[2]) )  ]  )
    #displayy(path,nameToDisp+"_orig")
    axis = 2
    pre_slices = origSize[axis]
    post_slices = new_size[axis]
    Z_FAC = post_slices/pre_slices # Sampling factor in Z direction
    thicks_ori=sitk.GetArrayFromImage(imageOrig)
    # Call the SR interpolation tool from KevinSR
    thins_gen = SOUP_GAN(thicks_ori, Z_FAC,1)
    image1 = sitk.GetImageFromArray(thins_gen)
    image1.SetSpacing((orig_spacing[0],orig_spacing[1],targetSpac[2])) #updating spacing
    image1.SetOrigin(imageOrig.GetOrigin())
    image1.SetDirection(imageOrig.GetDirection())    
    #displayy_image(image1,nameToDisp+"_resampled")
    
    
    
targetSpac=(0.3,0.3, 2)

testResample(pathT2w, "t2w",targetSpac)


1/1 [==============================] - 13s 13s/step


working in Z dir


In [ ]:
#pathT2w,pathHbv,pathADC,patht2wLabel
def testResample(path, nameToDisp,targetSpac):
    imageOrig = sitk.ReadImage(path)
    origSize= imageOrig.GetSize()
    orig_spacing=imageOrig.GetSpacing()
    print(f"origSize {origSize}")
    spacc=(0.3,0.3, 2)
    new_size = tuple([int(origSize[0]*(orig_spacing[0]/targetSpac[0])),
                    int(origSize[1]*(orig_spacing[1]/targetSpac[1])),
                    int(origSize[2]*(orig_spacing[2]/targetSpac[2]) )  ]  )
    print(f"new_size {new_size}")
    displayy(path,nameToDisp+"_orig")
    axis = 2
    pre_slices = origSize[axis]
    post_slices = new_size[axis]
    Z_FAC = post_slices/pre_slices # Sampling factor in Z direction
    thicks_ori=sitk.GetArrayFromImage(imageOrig)
    thicks_ori = np.moveaxis(thicks_ori, 0, 2)
    
    image1=copyDirAndOrigin(imageOrig,orig_spacing,thicks_ori)
    displayy_image(image1,nameToDisp+"_transposed")    
    
    #Call the SR interpolation tool from KevinSR
    print(f"thicks_ori shape {thicks_ori.shape} ")

    thins_gen = SOUP_GAN(thicks_ori, Z_FAC,1)
    print(f"thins_gen shape {thins_gen.shape} ")
    thins_gen = np.moveaxis(thins_gen, 2, 0)
    
    image1=copyDirAndOrigin(imageOrig,(orig_spacing[0],orig_spacing[1],targetSpac[2]),thins_gen)
    displayy_image(image1,nameToDisp+"_resampled")
    return (imageOrig,image1)
    
    
targetSpac=(0.3,0.3, 2)

imageee= testResample(pathT2w, "t2w",targetSpac)

working x axis


In [ ]:
import pandas as pd
import SimpleITK as sitk
from KevinSR import mask_interpolation, SOUP_GAN
import os
import numpy as np
from scipy import ndimage, interpolate
from scipy.ndimage import zoom
from KevinSR import SOUP_GAN
import monai
import SimpleITK as sitk
from copy import deepcopy
import tensorflow as tf

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
row= df.iloc[19]
pathT2w= row['t2w']
pathsag= row['sag']

pathHbv= row['hbv']
pathADC= row['adc']
patht2wLabel= row['reSampledPath']
testImagePath = '/home/sliceruser/data/exploration/resampledd.mha'

def copyDirAndOrigin(imageOrig,spacing,data):
    image1 = sitk.GetImageFromArray(data)
    image1.SetSpacing(spacing) #updating spacing
    image1.SetOrigin(imageOrig.GetOrigin())
    image1.SetDirection(imageOrig.GetDirection()) 
    print(image1.GetSize())
    return image1

#pathT2w,pathHbv,pathADC,patht2wLabel
def testResample(path, nameToDisp,targetSpac):
    imageOrig = sitk.ReadImage(path)
    origSize= imageOrig.GetSize()
    orig_spacing=imageOrig.GetSpacing()
    print(f"origSize {origSize}")
    new_size = tuple([int(origSize[0]*(orig_spacing[0]/targetSpac[0])),
                    int(origSize[1]*(orig_spacing[1]/targetSpac[1])),
                    int(origSize[2]*(orig_spacing[2]/targetSpac[2]) )  ]  )
    print(f"new_size {new_size}")
    displayy(path,nameToDisp+"_orig")
    axis = 0
    pre_slices = origSize[axis]
    post_slices = new_size[axis]
    Z_FAC = post_slices/pre_slices # Sampling factor in Z direction
    thicks_ori=sitk.GetArrayFromImage(imageOrig)
    #thicks_ori = np.moveaxis(thicks_ori, 1, 2)
    
    #Call the SR interpolation tool from KevinSR
    print(f"thicks_ori shape {thicks_ori.shape} ")

    thins_gen = SOUP_GAN(thicks_ori, Z_FAC,1)
    print(f"thins_gen shape {thins_gen.shape} ")
    #thins_gen = np.moveaxis(thins_gen, 2, 1)
    
    image1=copyDirAndOrigin(imageOrig,(targetSpac[0],orig_spacing[1], orig_spacing[2]),thins_gen)
    displayy_image(image1,nameToDisp+"_resampled")
    
    
    
    return (imageOrig,image1)
    
    
targetSpac=(0.3,0.3, 2)

imageee= testResample(pathsag, "sag",targetSpac)

probably work in Y


In [ ]:
def copyDirAndOrigin(imageOrig,spacing,data):
    image1 = sitk.GetImageFromArray(data)
    image1.SetSpacing(spacing) #updating spacing
    image1.SetOrigin(imageOrig.GetOrigin())
    image1.SetDirection(imageOrig.GetDirection()) 
    print(image1.GetSize())
    return image1

#pathT2w,pathHbv,pathADC,patht2wLabel
def testResample(path, nameToDisp,targetSpac):
    imageOrig = sitk.ReadImage(path)
    origSize= imageOrig.GetSize()
    orig_spacing=imageOrig.GetSpacing()
    print(f"origSize {origSize}")
    new_size = tuple([int(origSize[0]*(orig_spacing[0]/targetSpac[0])),
                    int(origSize[1]*(orig_spacing[1]/targetSpac[1])),
                    int(origSize[2]*(orig_spacing[2]/targetSpac[2]) )  ]  )
    print(f"new_size {new_size}")
    displayy(path,nameToDisp+"_orig")
    axis = 0
    pre_slices = origSize[axis]
    post_slices = new_size[axis]
    Z_FAC = post_slices/pre_slices # Sampling factor in Z direction
    thicks_ori=sitk.GetArrayFromImage(imageOrig)
    thicks_ori = np.moveaxis(thicks_ori, 1, 2)
    
    #Call the SR interpolation tool from KevinSR
    print(f"thicks_ori shape {thicks_ori.shape} ")

    thins_gen = SOUP_GAN(thicks_ori, Z_FAC,1)
    print(f"thins_gen shape {thins_gen.shape} ")
    thins_gen = np.moveaxis(thins_gen, 2, 1)
    
    image1=copyDirAndOrigin(imageOrig,(orig_spacing[0],targetSpac[1], orig_spacing[2]),thins_gen)
    displayy_image(image1,nameToDisp+"_resampled")
    
    
    
    return (imageOrig,image1)
    
    
targetSpac=(0.3,0.3, 2)

imageee= testResample(pathsag, "sag",targetSpac)